### Importing

In [ ]:
import os
import numpy as np
import tensorflow as tf

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import TimeDistributed, GRU, Flatten, CuDNNLSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from sklearn.metrics import accuracy_score

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

### Getting and preparation data

In [ ]:
!git clone https://github.com/antpal/pedestrian_activity_recognition

In [3]:
classes_dict = {'0': 'backward', '1':'forward', '2':'left', '3':'right', '4':'on_place'}
ds_path = 'pedestrian_activity_recognition/poses/'

In [4]:
def load_pose_estimation(path, filename):
    """
    Retrieving pose estimation tensors from a CSV file
    """
    loaded_arr = np.loadtxt(os.path.join(path, filename))
    # This loadedArr is a 2D array, therefore we need to convert it to the original array shape.
    # Reshaping to get original matrice with original shape.
    loaded_original = loaded_arr.reshape(loaded_arr.shape[0], loaded_arr.shape[1] // 3, 3)
    return loaded_original

In [5]:
def process_load_pose_estimation(path):
  """
  Retrieveing pose estimation tensors from all CSV files in a folder (path)
  """
  poses_array = []
  for filename in os.listdir(path):
    if filename.endswith(".csv"): 
        #print(os.path.join(path, filename))
        poses_array.append(load_pose_estimation(path, filename))
        continue
    else:
        continue
  return poses_array

In [6]:
poses_train = []
classes_train = []

for c in classes_dict.items():
  poses_train_temp = []
  poses_train_temp = process_load_pose_estimation(ds_path + 'train/' + c[1] + '/')
  poses_train = np.append(poses_train, poses_train_temp)
  classes_train = np.append(classes_train, np.array([c[0]]*len(poses_train_temp)))

poses_train = np.reshape(poses_train, (-1, 80, 17, 3))
classes_train = classes_train.astype(int)

In [7]:
poses_test = []
classes_test = []

for c in classes_dict.items():
  poses_test_temp = []
  poses_test_temp = process_load_pose_estimation(ds_path + 'test/' + c[1] + '/')
  poses_test = np.append(poses_test, poses_test_temp)
  classes_test = np.append(classes_test, np.array([c[0]]*len(poses_test_temp)))

poses_test = np.reshape(poses_test, (-1, 80, 17, 3))
classes_test = classes_test.astype(int)

In [ ]:
print(poses_train.shape)
print(classes_train.shape)
print(poses_test.shape)
print(classes_test.shape)

### Train NN

In [10]:
frames_count = 80
shape = (frames_count, 17, 3)
batch_size_model = 100
n_epochs = 100
optimizer = tf.keras.optimizers.Adam()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=5, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [18]:
model = Sequential()

model.add(
    TimeDistributed(
        Flatten(),
        input_shape=shape
    )
)
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dropout(0.2))
model.add(Dense(5))
#model.add(Activation('sigmoid'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

model.build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_1 (TimeDis  (None, 80, 51)           0         
 tributed)                                                       
                                                                 
 gru_2 (GRU)                 (None, 80, 128)           69504     
                                                                 
 gru_3 (GRU)                 (None, 64)                37248     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
 activation_1 (Activation)   (None, 5)                 0         
                                                      

In [11]:
classes_train_oh = np.array(tf.one_hot(classes_train, 5))
classes_test_oh = np.array(tf.one_hot(classes_test, 5))

In [ ]:
history = model.fit(
    poses_train, classes_train_oh, 
    batch_size=batch_size_model, 
    epochs=n_epochs,
    validation_data=(poses_test, classes_test_oh),
    verbose=2,
    #callbacks=[es, mc]
    callbacks=[mc]
)

### Inference

In [20]:
best_model = load_model('best_model.h5')

In [21]:
poses_predict = best_model.predict(poses_test)
poses_predict_classes=np.argmax(poses_predict, axis=-1)

In [ ]:
accuracy_score(classes_test, poses_predict_classes)